In [1]:
import pandas as pd
import numpy as np
import warnings
import os

warnings.filterwarnings('ignore')

# ==========================================
# 1. Configuration (Submission Ready)
# ==========================================
class Config:
    # Quarters for submission
    QUARTERS = ['2023_Q1', '2023_Q3', '2023_Q4', '2024_Q2', '2024_Q4', '2025_Q1', '2025_Q2']
    
    # Target Asset (AUD)
    ASSETS = ['AUD'] 
    
    # Contract Specs (Group 2: AUD)
    # Transaction Cost $10, Point Value $100,000
    SPECS = {
        'AUD': {'point_val': 100000.0, 'cost': 10.0}
    }

    # === Trading Hours Rules (Group 2 Strict) ===
    # Break: 17:00 - 18:00
    # Mandatory Exit: 16:50 (10 mins before break)
    # Restart Trading: 18:10 (10 mins after break)
    EXIT_TIME = pd.to_datetime("16:50").time()
    RESTART_TIME = pd.to_datetime("18:10").time()

    # === Strategy Parameters (Optimized for Quarterly Reset) ===
    # 5-minute data frequency
    # Window adjusted to 96 bars (approx 8 hours) to allow early trading in the quarter.
    
    BB_WINDOW = 96     # 8 Hours lookback
    BB_SIGMA = 2.0     # Standard deviation threshold
    
    # Annualization factor
    ANNUALIZATION = 252

def mySR(daily_pnl, scale=252):
    """Calculates Annualized Sharpe Ratio based on Daily PnL"""
    if np.nanstd(daily_pnl) == 0: return 0
    return np.sqrt(scale) * np.nanmean(daily_pnl) / np.nanstd(daily_pnl)

def find_data_path(quarter, prefix='data2'):
    """Helper to find data in various relative paths"""
    filename = f'{prefix}_{quarter}.parquet'
    potential_paths = [
        f'data/{filename}',          # Same folder
        f'../data/{filename}',       # Up one level
        f'../../data/{filename}',    # Up two levels
        f'../../../data/{filename}', # Up three levels
        filename                     # Current folder
    ]
    for path in potential_paths:
        if os.path.exists(path): return path
    return None

# ==========================================
# 2. Strategy Engine (Per-Quarter Logic)
# ==========================================
print("\n--- Calculating Group 2 (AUD Bollinger Bands) Stats ---")
results = []

for quarter in Config.QUARTERS:
    try:
        # --- 1. Data Loading ---
        file_path = find_data_path(quarter, prefix='data2')
        if file_path is None:
            raise FileNotFoundError(f"File not found for {quarter}")

        df = pd.read_parquet(file_path)
        df.set_index('datetime', inplace=True)

        # --- 2. Data Cleaning & Time Filtering ---
        # Remove break data (17:00-18:00) strictly
        df.loc[df.between_time("17:00", "18:00").index] = np.nan
        
        # --- 3. Indicators Calculation ---
        price = df['AUD']
        
        # Calculate Bollinger Bands
        # min_periods=BB_WINDOW ensures stability, but short enough for quarterly start
        ma = price.rolling(window=Config.BB_WINDOW, min_periods=Config.BB_WINDOW).mean()
        std = price.rolling(window=Config.BB_WINDOW, min_periods=Config.BB_WINDOW).std()
        
        upper = ma + (Config.BB_SIGMA * std)
        lower = ma - (Config.BB_SIGMA * std)
        
        # --- 4. Iterative Execution Loop ---
        times = df.index.time
        price_vals = price.values
        upper_vals = upper.values
        lower_vals = lower.values
        ma_vals = ma.values
        
        pos_aud = np.zeros(len(df))
        curr_pos = 0
        
        for i in range(len(df)):
            t = times[i]
            p = price_vals[i]
            
            # --- Rule: Mandatory Exit at 16:50 ---
            if t >= Config.EXIT_TIME and t < Config.RESTART_TIME:
                curr_pos = 0
                pos_aud[i] = 0
                continue
                
            # Skip if indicators are NaN
            if np.isnan(upper_vals[i]) or np.isnan(lower_vals[i]):
                pos_aud[i] = curr_pos
                continue

            u = upper_vals[i]
            l = lower_vals[i]
            m = ma_vals[i]

            # --- Strategy Logic: Mean Reversion ---
            if curr_pos == 0:
                if p > u:
                    # Price above Upper Band -> Short
                    curr_pos = -1
                elif p < l:
                    # Price below Lower Band -> Long
                    curr_pos = 1
            
            elif curr_pos == 1: # Long
                # Exit if price recovers to MA (or goes higher)
                if p >= m:
                    curr_pos = 0
                    
            elif curr_pos == -1: # Short
                # Exit if price drops to MA (or goes lower)
                if p <= m:
                    curr_pos = 0
            
            pos_aud[i] = curr_pos

        # --- 5. PnL Calculation ---
        # Shift positions by 1
        pos_held = pd.Series(pos_aud, index=df.index).shift(1).fillna(0)
        
        # SAFETY CHECK: Ensure positions are 0 during forbidden times
        mask_forbidden = (pd.Series(df.index, index=df.index).dt.time >= Config.EXIT_TIME) & \
                         (pd.Series(df.index, index=df.index).dt.time < Config.RESTART_TIME)
        pos_held[mask_forbidden] = 0

        # Calculate PnL (Gross)
        pnl_gross = pos_held * df['AUD'].diff() * Config.SPECS['AUD']['point_val']
        
        # Calculate Costs
        trades = pos_held.diff().abs().fillna(0)
        total_cost = trades * Config.SPECS['AUD']['cost']
        
        pnl_net = pnl_gross - total_cost
        
        # --- 6. Aggregation & Metrics ---
        daily_net = pnl_net.resample('D').sum()
        # Remove non-trading days
        daily_net = daily_net[daily_net != 0] 
        
        net_sr = mySR(daily_net, 252)
        net_cum = daily_net.sum()
        
        # === PROFESSOR'S STATISTIC FORMULA ===
        abs_pnl_scaled = abs(net_cum) / 1000.0
        log_term = np.log(abs_pnl_scaled) if abs_pnl_scaled > 0 else 0
        stat_val = (net_sr - 0.5) * max(0, log_term)

        results.append({
            'Quarter': quarter,
            'Net SR': round(net_sr, 2),
            'Net PnL': round(net_cum, 2),
            'Trades/Day': round(trades.resample('D').sum().mean(), 2),
            'Stat': round(stat_val, 2)
        })

    except Exception as e:
        print(f"Skipping {quarter}: {e}")

# ==========================================
# 3. Final Report (Console Only)
# ==========================================
df_res = pd.DataFrame(results)
print("\n=== Group 2 (AUD) Results ===")
print(df_res.to_markdown(index=False))

if not df_res.empty:
    print(f"\nTotal Stat: {df_res['Stat'].sum():.2f}")


--- Calculating Group 2 (AUD Bollinger Bands) Stats ---

=== Group 2 (AUD) Results ===
| Quarter   |   Net SR |   Net PnL |   Trades/Day |   Stat |
|:----------|---------:|----------:|-------------:|-------:|
| 2023_Q1   |     1.64 |      2496 |         3.18 |   1.04 |
| 2023_Q3   |    -1.06 |     -1223 |         2.87 |  -0.31 |
| 2023_Q4   |    -0.8  |      -903 |         3.36 |  -0    |
| 2024_Q2   |    -5.1  |     -5218 |         2.84 |  -9.26 |
| 2024_Q4   |     1    |       957 |         2.96 |   0    |
| 2025_Q1   |    -2.67 |     -2286 |         2.78 |  -2.62 |
| 2025_Q2   |    -1.22 |     -1442 |         3.01 |  -0.63 |

Total Stat: -11.78
